In [33]:
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

In [1]:
spark

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [3]:
images_folders = ['/CA1/Images/Ireland/','/CA1/Images/Honduran/']

In [4]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [5]:
images_rdd

org.apache.spark.api.java.JavaPairRDD@7db99533

In [6]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [7]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [8]:
pandasImagesDF = imageDf.toPandas()

In [9]:
IMG_SHAPE=225

In [10]:
def processImage(data, target_size=(IMG_SHAPE,IMG_SHAPE)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(new_image).reshape([target_size[0],target_size[1],3])
        return array

In [11]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [ ]:
# pandasImagesDF

In [12]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [13]:
pandasImagesDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    180 non-null    object
 1   Label   180 non-null    object
 2   Data    180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [14]:
NCATEGORIES = len(pandasImagesDF["Label"].unique())

Importing from Keras functionality necessary to implement CNN

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Now we are to split the data using train_test_split function to use Kfold and improve the perfomance of the model

In [89]:
X_train, X_test, y_train, y_test = train_test_split(pandasImagesDF["Data"],pandasImagesDF["Label"],test_size=0.3, random_state=42)

In [90]:
from sklearn.preprocessing import OneHotEncoder

In [91]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [92]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

In [93]:
X_train = np.reshape(X_train,(X_train.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_test = np.reshape(X_test,(X_test.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)

Because the pixels are from 0 to 255, We have to normalize the pixels

In [94]:
X_train/=255
X_test/=255

In [95]:
X_train.shape,X_test.shape

((126, 225, 225, 3), (54, 225, 225, 3))

In [96]:
y_train.shape, y_test.shape

((126, 2), (54, 2))

In [97]:
model = Sequential()
model.add(Input(shape=(225, 225, 3)))  # 225x225 RGB images
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
model.add(MaxPool2D(3))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))

model.add(Flatten())


model.add(Dense(128, activation="relu"))
model.add(Dense(NCATEGORIES,activation="sigmoid"))

model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

2024-03-21 22:34:37.134925: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 22:34:37.135580: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [99]:
model.fit(X_train, y_train, batch_size=50, epochs=35, verbose=1, validation_data=(X_test, y_test))

Epoch 1/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5284 - loss: 0.6921 - val_accuracy: 0.4259 - val_loss: 0.6848
Epoch 2/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5409 - loss: 0.6918 - val_accuracy: 0.4259 - val_loss: 0.6746
Epoch 3/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5409 - loss: 0.6788 - val_accuracy: 0.4444 - val_loss: 0.6856
Epoch 4/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5363 - loss: 0.6839 - val_accuracy: 0.4630 - val_loss: 0.6814
Epoch 5/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5488 - loss: 0.6804 - val_accuracy: 0.4630 - val_loss: 0.6730
Epoch 6/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5888 - loss: 0.6641 - val_accuracy: 0.5556 - val_loss: 0.6940
Epoch 7/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6295 - loss: 0.6675 - val_accuracy: 0.5370 - val_loss: 0.7089
Epoch 8/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6738 - loss: 0.6476 - val_accuracy: 0.5185 - val_loss: 0.7012
Epoch 9/35
3/3 ━

In [100]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 223, 223, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 221, 221, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 69, 69, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 34, 34, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 11, 11, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       102,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 505,256 (1.93 MB)

 Trainable params: 168,418 (657.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 336,838 (1.28 MB)

In [102]:
pred = model.predict(X_test[:25])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


In [103]:
def comparePredictions(predictionsValues, expectedValues):
    correct = 0
    for x in range(len(predictionsValues)):
        if predictionsValues[x] == expectedValues[x]:
            correct+=1
    
    print(f'Correct predictions {correct}/{len(predictionsValues)}. Percent {correct/len(predictionsValues)}')

In [104]:
comparePredictions(np.argmax(pred, axis=1),np.argmax(y_test[:25], axis=1))

Correct predictions 11/25. Percent 0.44
